# QnA over PDF
Reference from https://www.youtube.com/watch?v=TLf90ipMzfE

In [1]:
!pip install langchain
!pip install faiss-cpu
!pip install openai
!pip install PyPDF2

     ---------------------------------------- 0.0/626.5 kB ? eta -:--:--
      --------------------------------------- 10.2/626.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/626.5 kB 262.6 kB/s eta 0:00:03
     -- ---------------------------------- 41.0/626.5 kB 281.8 kB/s eta 0:00:03
     ----- ------------------------------- 92.2/626.5 kB 479.1 kB/s eta 0:00:02
     -------- --------------------------- 143.4/626.5 kB 610.6 kB/s eta 0:00:01
     ------------- ---------------------- 235.5/626.5 kB 850.1 kB/s eta 0:00:01
     ------------------------ ------------- 399.4/626.5 kB 1.2 MB/s eta 0:00:01
     -------------------------------------  614.4/626.5 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 626.5/626.5 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ----------- ---------------------------- 0.6/2.1 MB 13.1 MB/s eta 0:00:01
     ------------------ --------------------- 1.0/2.1 MB 10.5 M

     ---------------------------------------- 0.0/70.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/70.3 kB ? eta -:--:--
     ---------------- --------------------- 30.7/70.3 kB 259.2 kB/s eta 0:00:01
     --------------------------------- ---- 61.4/70.3 kB 409.6 kB/s eta 0:00:01
     -------------------------------------- 70.3/70.3 kB 425.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
     - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
     ---- -------------------------------- 30.7/232.6 kB 220.2 kB/s eta 0:00:01
     ------ ------------------------------ 41.0/232.6 kB 245.8 kB/s eta 0:00:01
     ----------- ------------------------- 71.7/232.6 kB 359.3 kB/s eta 0:00:01
     ---------------------- ------------- 143.4/232.6 kB 568.9 kB/s eta 0:00:01
     ------------------------------- ---- 204.8/232.6 kB 692.4 kB/s eta 0:00:01
     ------------------------------------ 232.6/232.6 kB 751.1 kB/s 

In [4]:
with open('./keys/openai.txt', 'r') as fp:
    key = fp.readline()

In [7]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [6]:
import os
os.environ['OPENAI_API_KEY'] = key

In [9]:
reader = PdfReader('./pdf_docs/gpt4.pdf')

In [11]:
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text+=text

In [17]:
text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [19]:
len(texts)

357

In [23]:
print(texts[0], '\n----------------------------\n', texts[1])

GPT-4 Technical Report
OpenAI
Abstract
We report the development of GPT-4, a large-scale, multimodal model which can
accept image and text inputs and produce text outputs. While less capable than
humans in many real-world scenarios, GPT-4 exhibits human-level performance
on various professional and academic benchmarks, including passing a simulated
bar exam with a score around the top 10% of test takers. GPT-4 is a Transformer-
based model pre-trained to predict the next token in a document. The post-training
alignment process results in improved performance on measures of factuality and
adherence to desired behavior. A core component of this project was developing
infrastructure and optimization methods that behave predictably across a wide
range of scales. This allowed us to accurately predict some aspects of GPT-4’s
performance based on models trained with no more than 1/1,000th the compute of
GPT-4.
1 Introduction 
----------------------------
 range of scales. This allowed us to 

In [31]:
embeddings = OpenAIEmbeddings(disallowed_special=())

In [32]:
docsearch = FAISS.from_texts(texts, embeddings)

In [44]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [54]:
chain = load_qa_chain(ChatOpenAI(model_name='gpt-3.5-turbo'), chain_type = 'map_reduce', return_intermediate_steps=True)

In [57]:
query = 'How does the model do in accuracy?'
docs = docsearch.similarity_search(query)
# chain.run(input_documents = docs, question=query)
chain({'input_documents' : docs, 'question':query}, return_only_outputs=True)

{'intermediate_steps': ['There is information on accuracy in the given text. The text states that the accuracy for an earlier version was around 30%, but a newer version increased accuracy to around 60%. Additionally, Figure 8 shows the performance of GPT-4 on TruthfulQA, with accuracy shown on the y-axis.',
  "According to Figure 6, the accuracy of the model is shown on the y-axis, with higher being better. An accuracy of 1.0 means the model's answers are judged to be in agreement with human ideal responses for all questions in the evaluation. GPT-4 improves on the latest GPT-3.5 model by 19 percentage points, with significant gains across all topics. Additionally, GPT-4 makes progress on public benchmarks like TruthfulQA, which tests the model's ability to separate fact from an adversarially-selected set of incorrect statements. However, it is only slightly better at this task than GPT-3.5, but after RLHF post-training, large improvements are observed.",
  'The document mentions the 